In [1]:
# data download
# wget -P A B : B의 데이터를 A에 저장.
!wget -P data https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt # train data 
!wget -P data https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt # test data

--2017-06-02 16:15:12--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com... 151.101.72.133
Connecting to raw.githubusercontent.com|151.101.72.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: 'data/ratings_train.txt.1'

ratings_train.txt.1 100%[===================>]  13.95M  31.3KB/s    in 8m 28s  

2017-06-02 16:23:43 (28.1 KB/s) - 'data/ratings_train.txt.1' saved [14628807/14628807]

--2017-06-02 16:23:43--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com... 151.101.72.133
Connecting to raw.githubusercontent.com|151.101.72.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: 'data/ratings_test.txt.1'

ratings_test.txt.1  100%[===================>]   4.67M  31.4KB/s    in 93s     

2017-06-02 16:25:18 (51.5 KB/s) - 'data/ratings_test.txt.1' saved 

In [1]:
import re
import numpy as np

# Load Pre-trained Word2vec Model
#from gensim.models import word2vec

# Data visualization
from matplotlib import pyplot as plt
%matplotlib inline

# CNN
#from keras.preprocessing.text import Tokenizer, text_to_word_sequence
#from keras.preprocessing.sequence import pad_sequences
#from keras import losses

#from keras.layers import Embedding
#from keras.layers import Dense, Input, Flatten
#from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout
#from keras.models import Model, Sequential

In [2]:
TRAIN_FILE = 'data/ratings_train.txt'
TEST_FILE = 'data/ratings_test.txt'

In [3]:
!head data/ratings_train.txt

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1


In [12]:

reviews = []
with open(TRAIN_FILE, 'r') as f:
    next(f) # header skip
    for line in f.readlines():
        _, review, _ = line.strip().split('\t')
        reviews.append(review.strip())
        
with open(TEST_FILE, 'r') as f:
    next(f) # header skip
    for line in f.readlines():
        _, review, _ = line.strip().split('\t')
        reviews.append(review.strip())

In [13]:
reviews[:10]

['아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
 '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.',
 '원작의 긴장감을 제대로 살려내지못했다.',
 '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
 '액션이 없는데도 재미 있는 몇안되는 영화',
 '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?']

In [ ]:
%%time
train_reviews = [(tokenizer(review), int(label)) for review, label in train_reviews]
test_reviews = [(tokenizer(review), int(label)) for review, label in test_reviews]

In [14]:
from gensim.models import word2vec

In [15]:
from konlpy.tag import Twitter
twi_tagger = Twitter()

In [16]:
sentence = "자연어 처리란 무엇일까까까깎ㄲㄲ"
twi_tagger.pos(sentence,norm=True,stem=True)

[('자연어', 'Noun'),
 ('처리', 'Noun'),
 ('란', 'Noun'),
 ('무엇', 'Noun'),
 ('일', 'Noun'),
 ('끄다', 'Verb'),
 ('깎다', 'Verb'),
 ('ㄲㄲ', 'KoreanParticle')]

# Word Enbedding 

* 단어를 벡터에 임베딩함
* CNN 학습시 필요 
* Word2vec은 gensim을 통해 쉽게 만들수 있음  

In [17]:
import os
import sys
import multiprocessing
import numpy as np

#토큰화
from konlpy.tag import Twitter
twi_tagger = Twitter()

#word2vec 모델링
import gensim
from gensim.models import word2vec

#차원 축소 및 시각화 
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show, save , output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from bokeh import palettes
output_notebook()
%matplotlib inline

Loading BokehJS ...

In [18]:
def tokenizer(doc):
    tokens = ['/'.join(t)for t in twi_tagger.pos(doc,norm=True, stem = True)]
    return tokens

In [19]:
tokenizer(reviews[1])

['흠/Noun',
 '.../Punctuation',
 '포스터/Noun',
 '보고/Noun',
 '초딩/Noun',
 '영화/Noun',
 '줄/Noun',
 '..../Punctuation',
 '오버/Noun',
 '연기/Noun',
 '조차/Josa',
 '가볍다/Adjective',
 '않다/Verb']

In [20]:
%%time
# Tokenize on every reviews.
reviews = [tokenizer(review) for review in reviews]

CPU times: user 5min 42s, sys: 3.29 s, total: 5min 45s
Wall time: 5min 36s


In [21]:
#Set parameters
num_features = 300 # word vector dimensionality
min_word_count = 1 #Minimum word count
num_workers = multiprocessing.cpu_count() # number of threads to run in parallel
context = 4 #context size

iter_  = 100 # Iteration
sg = 1 # Skip-gram or not

model_name = '{}dim_{}minwords_{}context_twi'.format(num_features,min_word_count,context)

In [22]:
%%time
#train a word2vec model
model = word2vec.Word2Vec(reviews,workers= num_workers,\
                         size = num_features, min_count= min_word_count, \
                         window = context , iter = iter_,sg = sg
                         )

CPU times: user 1h 27min 5s, sys: 26.4 s, total: 1h 27min 32s
Wall time: 24min 44s


In [26]:
# Save a Word2vec model
model_file = os.path.join('w2v', model_name)
model_file

'w2v/300dim_1minwords_4context_twi'

In [28]:
model.save('./w2v')

In [29]:
def vis_top_n(model, top, lr=500, n_iter=1000, perplexity=10):
    vectors = model.wv.syn0[:top]
    labels = model.wv.index2word[:top]

    if np.shape(vectors)[1] > 2:
        tsne = TSNE(perplexity=perplexity, n_components=2, init='random', n_iter=n_iter, verbose=1, learning_rate=lr, method='exact')  # tsne를 이용한 차원 축소 (n차원 -> 2차원)
        vectors = tsne.fit_transform(vectors)

    source = ColumnDataSource(
        data=dict(
            x=vectors.T[0],
            y=vectors.T[1],
            word=labels,
        )
    )
    label_set = LabelSet(x='x', y='y', text='word', level='glyph', text_color="#111111", text_alpha=0.6, text_font_size='8pt', x_offset=5, y_offset=5, source=source, render_mode='canvas')
    tools = "pan,wheel_zoom,box_zoom,reset,resize"
    p = figure(plot_width=900, plot_height=900, tools=[tools], title='word2vec vis top %i' % top)
    p.circle('x', 'y', size=5, source=source, alpha=0.6, fill_color='red', line_color="#ff9900")
    p.add_layout(label_set)
    show(p)

In [30]:
vis_top_n(model=model, top=2000, lr=2000, n_iter=1000, perplexity=30)

[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 1000 / 2000
[t-SNE] Computed conditional probabilities for sample 2000 / 2000
[t-SNE] Mean sigma: 1.356784
[t-SNE] KL divergence after 100 iterations with early exaggeration: 34.969561
[t-SNE] Error after 225 iterations: 34.969561
